# Pré seleção - Implantação

Remove atributos de acordo com os seguintes critérios: 
- Variabilidade próximo de 0
- Alta correlação entre si
- Lida com NaN e valores faltantes

Este notebook apresenta:
- como carregar modelos e outros resultados do treinamento.
- como utilizar um modelo para fornecer predições em tempo real.

## Chamada do Transformer Personalizado

## Declaração de Classe para Predições em Tempo Real

A tarefa de implantação cria um serviço REST para predições em tempo real.<br>
Para isso você deve criar uma classe `Model` que implementa o método `predict`.

In [ ]:
%%writefile Model.py
import joblib
import numpy as np
import pandas as pd


class Model(object):
    def __init__(self):
        # Carrega artefatos: estimador, etc
        artifacts = joblib.load("/tmp/data/pre-selection.joblib")
        self.pipeline = artifacts["pipeline"]
        self.features_names_training = artifacts["columns"]
        self.features_after_pipeline = artifacts["features_after_pipeline"]

    def class_names(self):
        return self.features_after_pipeline.tolist()

    def predict(self, X, feature_names, meta=None):
        
        df = pd.DataFrame(X)
        
        # Antes de utilizar o conjunto de dados X no modelo, reordena suas features de acordo com a ordem utilizada no treinamento
        if feature_names:
            df = pd.DataFrame(X, columns=feature_names)
            X = df[self.features_names_training].to_numpy()

        # Realiza transformação
        X = self.pipeline.transform(X)

        return X